In [1]:
import numpy as np
import pandas as pd
import nltk
from fuzzywuzzy import fuzz
from nltk.metrics import jaccard_distance
from numpy import save
import itertools

/home/ig5eq/.local/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Setting up inputs for NDC x FDA

In [2]:
ndc = pd.read_csv("../data/working/ndc_clean.csv", index_col = [0])

In [3]:

ndc.dropna(inplace = True)
ndc.reset_index(inplace = True)

In [4]:
ndc_row = ndc['row']

In [5]:
ndc_org = ndc['original_company']

In [6]:
ndc_clean = ndc['cleaned_name']

In [7]:
fda = pd.read_csv("../data/working/fda_clean.csv", index_col = [0])

In [8]:
fda.dropna(inplace = True)
fda.reset_index(inplace = True)

In [9]:
fda.columns

Index(['index', 'FDA Companies ', 'Company Clean'], dtype='object')

In [10]:
fda_row = fda['index']

In [11]:
fda_og = fda['FDA Companies ']

In [12]:
fda_clean = fda['Company Clean']

### Function for matching 

In [13]:
##Function that takes in 3 lists, the list of strings you want to compare, the list of strings you're comparing to and a list with the original way the string from list 1 was written.  
def matchWithFuzzyNamesOG(l1, l2, og1, og2, row_num1, row_num2):
    match = pd.DataFrame(columns = ['clean fda company name', 'original fda company', 'fda row', 'clean ndc row', 'clean ndc company', 'original ndc company', 'fuzz ratio'])
    for (i, o, r) in zip(l1, og1, row_num1):
        matches = []
        score = []
        #score_partial = []
        row = []
        matchesog = []
        for (j, q, s) in zip(l2, og2, row_num2):
            #due to time constraints, we set the fuzz.ratio to 100, however future programming should adjust this value to some other quantity to see if we get better matches
            if (fuzz.ratio(i,j)) == 100:
                matches.append(j)
                matchesog.append(q)
                score.append(fuzz.ratio(i,j))
                row.append(s)
                #the partial ratio will return the ratio raw score of 100  if the shorter string is found an any of substrings of the larger string, used this as a safety net and potentially as an additional step to 
               # score_partial.append((fuzz.partial_ratio(i,j)))
                match = match.append({ 'clean fda company name': i, 'original fda company': o, 'fda row': r, 'clean ndc row': row, 'clean ndc company': matches,   'original ndc company': matchesog, 'fuzz ratio': score,},ignore_index=True)
        
#Will output a pandas dataframe with 4 columns
    return match


In [39]:
matchWithFuzzyNamesOG(fda_clean, ndc_clean, fda_og, ndc_org, fda_row, ndc_row)

,clean fda company name,original fda company,fda row,clean ndc row,clean ndc company,original ndc company,fuzz ratio
0,d imaging drug,3D IMAGING DRUG,0,[],[],[],[]
1,m,3M,1,"[20, 22]","[m, m]","[3M Company, 3M Health Care]","[100, 100]"
2,m drug delivery,3M DRUG DELIVERY,2,[],[],[],[]
3,aaipharma,AAIPHARMA LLC,3,[],[],[],[]
4,abbott,ABBOTT LABS,4,[],[],[],[]
...,...,...,...,...,...,...,...
968,zo skin,ZO SKIN HEALTH,970,[7009],[zo skin],"[ZO Skin Health, Inc.]",[100]
969,zydus,ZYDUS HLTHCARE,971,"[7017, 7018, 7019, 7020]","[zydus, zydus, zydus, zydus]","[Zydus Pharmaceuticals (USA) Inc., Zydus Pharm...","[100, 100, 100, 100]"
970,zydus,ZYDUS PHARMS USA,972,"[7017, 7018, 7019, 7020]","[zydus, zydus, zydus, zydus]","[Zydus Pharmaceuticals (USA) Inc., Zydus Pharm...","[100, 100, 100, 100]"
971,zydus,ZYDUS PHARMS USA INC,973,"[7017, 7018, 7019, 7020]","[zydus, zydus, zydus, zydus]","[Zydus Pharmaceuticals (USA) Inc., Zydus Pharm...","[100, 100, 100, 100]"


In [14]:
fdc_match = matchWithFuzzyNamesOG(fda_clean, ndc_clean, fda_og, ndc_org, fda_row, ndc_row)

In [64]:
fdc_match

,clean fda company name,original fda company,fda row,clean ndc row,clean ndc company,original ndc company,fuzz ratio
0,m,3M,1,"[20, 22]","[m, m]","[3M Company, 3M Health Care]","[100, 100]"
2,abbvie,ABBVIE,5,[66],[abbvie],[AbbVie Inc.],[100]
3,abbvie,ABBVIE INC,8,[66],[abbvie],[AbbVie Inc.],[100]
4,abraxis bioscience,ABRAXIS BIOSCIENCE,11,[71],[abraxis bioscience],"[Abraxis BioScience, LLC]",[100]
5,accord,ACCORD HLTHCARE,12,"[81, 82, 83, 84, 85]","[accord, accord, accord, accord, accord]","[Accord Healthcare, Accord Healthcare Inc, Acc...","[100, 100, 100, 100, 100]"
...,...,...,...,...,...,...,...
913,yaopharma,YAOPHARMA CO LTD,966,[6939],[yaopharma],"[Yaopharma Co., Ltd.]",[100]
914,zo skin,ZO SKIN HEALTH,970,[7009],[zo skin],"[ZO Skin Health, Inc.]",[100]
915,zydus,ZYDUS HLTHCARE,971,"[7017, 7018, 7019, 7020]","[zydus, zydus, zydus, zydus]","[Zydus Pharmaceuticals (USA) Inc., Zydus Pharm...","[100, 100, 100, 100]"
919,zydus,ZYDUS PHARMS USA,972,"[7017, 7018, 7019, 7020]","[zydus, zydus, zydus, zydus]","[Zydus Pharmaceuticals (USA) Inc., Zydus Pharm...","[100, 100, 100, 100]"


### Cleaning up the results to obtain Table with unique row #'s

In [15]:
fdc_match = fdc_match[~fdc_match.duplicated('fda row')]

In [16]:
fdc_match['clean fda company name'] = fdc_match['clean fda company name'].astype(str)
fdc_match['original fda company'] = fdc_match['original fda company'].astype(str)
fdc_match['fda row'] = fdc_match['fda row'].astype(str)

In [59]:
#ndc_unique = fdc_match[~fdc_match.duplicated('clean ndc company')]

TypeError: unhashable type: 'list'

In [69]:
#ndc_unique = fdc_match.drop_duplicates(subset='clean ndc company')

TypeError: unhashable type: 'list'

In [17]:
fdc_unique = fdc_match.groupby('clean fda company name', as_index = False).agg({'original fda company': ', '.join, 
                                                                            'fda row': ', '.join
                                                                                })

In [18]:
fdc_unique1 = fdc_match.groupby('clean fda company name', as_index = False).first()

In [22]:
fdc_unique[['corporate family', 'clean ndc row', 'clean ndc company', 'original ndc company', 'fuzz ratio']] = fdc_unique1[['clean fda company name','clean ndc row', 'clean ndc company', 'original ndc company', 'fuzz ratio']]

## DNA

In [24]:
dna = pd.read_csv("../data/working/dna_clean.csv", index_col = [0])

In [25]:
dna.dropna(inplace = True)
dna.reset_index(inplace = True)

In [26]:
dna.head(4)

,index,Unnamed: 0.1,Code,Description,cleaned_companies
0,0,0,AA,AA PLC,aa
1,1,2,AAAADJ,"Emperial Americas, Inc.",emperial americas
2,2,3,AAAAIY,"American Academy of Allergy, Asthma and Immuno...",academy allergy asthma immunology
3,3,5,AAABBB,Bird Studies Canada,bird studies canada


In [28]:
dna_row = dna['index']
dna_org = dna['Description']
dna_clean = dna['cleaned_companies']

In [29]:
dnf_match = matchWithFuzzyNamesOG(fda_clean, dna_clean, fda_og, dna_org, fda_row, dna_row)

In [31]:
dnf_match.to_csv('DNA and FDA matches')

In [24]:
dnf_match = pd.read_csv('DNA and FDA matches')

In [3]:
dnf_match

,Unnamed: 0,clean fda company name,original fda company,fda row,clean ndc row,clean ndc company,original ndc company,fuzz ratio
0,0,m,3M,1,"[33223, 37340, 38604, 53031, 56261, 58045]","['m', 'm', 'm', 'm', 'm', 'm']","['M101 Corp', '3M Company', 'M2 Group Ltd.', '...","[100, 100, 100, 100, 100, 100]"
1,1,m,3M,1,"[33223, 37340, 38604, 53031, 56261, 58045]","['m', 'm', 'm', 'm', 'm', 'm']","['M101 Corp', '3M Company', 'M2 Group Ltd.', '...","[100, 100, 100, 100, 100, 100]"
2,2,m,3M,1,"[33223, 37340, 38604, 53031, 56261, 58045]","['m', 'm', 'm', 'm', 'm', 'm']","['M101 Corp', '3M Company', 'M2 Group Ltd.', '...","[100, 100, 100, 100, 100, 100]"
3,3,m,3M,1,"[33223, 37340, 38604, 53031, 56261, 58045]","['m', 'm', 'm', 'm', 'm', 'm']","['M101 Corp', '3M Company', 'M2 Group Ltd.', '...","[100, 100, 100, 100, 100, 100]"
4,4,m,3M,1,"[33223, 37340, 38604, 53031, 56261, 58045]","['m', 'm', 'm', 'm', 'm', 'm']","['M101 Corp', '3M Company', 'M2 Group Ltd.', '...","[100, 100, 100, 100, 100, 100]"
...,...,...,...,...,...,...,...,...
613,613,wockhardt,WOCKHARDT LTD,953,[61416],['wockhardt'],['Wockhardt Ltd'],[100]
614,614,wockhardt,WOCKHARDT USA,954,[61416],['wockhardt'],['Wockhardt Ltd'],[100]
615,615,wyeth,WYETH PHARMS INC,957,"[1808, 15691]","['wyeth', 'wyeth']","['Wyeth LLC', 'Wyeth Ltd.']","[100, 100]"
616,616,wyeth,WYETH PHARMS INC,957,"[1808, 15691]","['wyeth', 'wyeth']","['Wyeth LLC', 'Wyeth Ltd.']","[100, 100]"


In [25]:
dnf_match = dnf_match[~dnf_match.duplicated('fda row')]

In [26]:
dnf_match

,Unnamed: 0,clean fda company name,original fda company,fda row,clean ndc row,clean ndc company,original ndc company,fuzz ratio
0,0,m,3M,1,"[33223, 37340, 38604, 53031, 56261, 58045]","['m', 'm', 'm', 'm', 'm', 'm']","['M101 Corp', '3M Company', 'M2 Group Ltd.', '...","[100, 100, 100, 100, 100, 100]"
6,6,aaipharma,AAIPHARMA LLC,3,[3820],['aaipharma'],['AAIPharma Services Corp.'],[100]
7,7,abbott,ABBOTT LABS,4,"[260, 273, 275]","['abbott', 'abbott', 'abbott']","['Abbott AG', 'Abbott Oy', 'Abbott Srl']","[100, 100, 100]"
10,10,abbvie,ABBVIE,5,[488],['abbvie'],['AbbVie Inc.'],[100]
11,11,abbvie,ABBVIE INC,8,[488],['abbvie'],['AbbVie Inc.'],[100]
...,...,...,...,...,...,...,...,...
612,612,wockhardt bio,WOCKHARDT BIO AG,952,[62283],['wockhardt bio'],['Wockhardt Bio AG'],[100]
613,613,wockhardt,WOCKHARDT LTD,953,[61416],['wockhardt'],['Wockhardt Ltd'],[100]
614,614,wockhardt,WOCKHARDT USA,954,[61416],['wockhardt'],['Wockhardt Ltd'],[100]
615,615,wyeth,WYETH PHARMS INC,957,"[1808, 15691]","['wyeth', 'wyeth']","['Wyeth LLC', 'Wyeth Ltd.']","[100, 100]"


In [27]:
dnf_match['clean fda company name'] = dnf_match['clean fda company name'].astype(str)
dnf_match['original fda company'] = dnf_match['original fda company'].astype(str)
dnf_match['fda row'] = dnf_match['fda row'].astype(str)

In [28]:
dnf_match_unique = dnf_match.groupby('clean fda company name', as_index = False).agg({'original fda company': ', '.join, 
                                                                            'fda row': ', '.join
                                                                                })

In [29]:
dnf_match1 = dnf_match.groupby('clean fda company name', as_index = False).first()

In [31]:
dnf_match_unique[['corporate family', 'clean dna row', 'clean dna company', 'original dna company', 'fuzz ratio']] = dnf_match1[['clean fda company name','clean ndc row', 'clean ndc company', 'original ndc company', 'fuzz ratio']]

In [33]:
dnf_match_unique.to_csv('../data/working/fda_dna_matching.csv')